_See [Readme](https://github.com/fleuryc/oc_ingenieur-ia_P2-Participez-a-un-concours-sur-la-Smart-City#readme) for installation instructions_

---


# Concours Data is for Good : aidons Paris à devenir une smart-city !

## Contexte

Dans le cadre du programme "Végétalisons la ville" organisé par la ville de Paris, nous proposons ici une analyse exploratoire des données OpenData concernant les arbres gérés par la ville de Paris.

L'objectif est d'aider Paris à devenir une "Smart-City" en gérant ses arbres de la manière la plus responsable possible. C'est-à-dire en optimisant les trajets nécessaires pour entretenir ces arbres.


## Outils utilisés

Nous allons utiliser le langage Python, et présenter ici le code, les résultats et l'analyse sous forme de [Notebook Jupyter](https://jupyterlab.readthedocs.io/en/stable/getting_started/overview.html).

Nous allons aussi utiliser les bibliothèques usuelles d'exploration et analyse de données, afin d'améliorer la simplicité et la performance de notre code :
* [NumPy](https://numpy.org/doc/stable/user/quickstart.html) et [Pandas](https://pandas.pydata.org/docs/user_guide/index.html) : effectuer des calculs scientifiques (statistiques, algèbre, ...) et manipuler des séries et tableaux de données volumineuses et complexes
* [Matplotlib](https://matplotlib.org/stable/tutorials/introductory/usage.html), [Pyplot](https://matplotlib.org/stable/tutorials/introductory/pyplot.html), [Seaborn](https://seaborn.pydata.org/tutorial/function_overview.html) et [Plotly](https://plotly.com/python/getting-started/) : générer des graphiques lisibles, intéractifs et pertinents


In [14]:
# Import libraries
import os.path
from zipfile import ZipFile as zf
import requests

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## If you use Notebook (and not JupyterLab), uncomment following lines
# import plotly.io as pio
# pio.renderers.default='notebook'


## Chargement des données et premier aperçu

Les données mises à disposition sont issues de  [opendata.paris.fr](https://opendata.paris.fr/explore/dataset/les-arbres/information/) et représentent "l’ensemble des arbres, ainsi que les arbres d’alignement, présents sur le territoire parisien et des cimetières extra-muros (hors de Paris)."



Nous allons dans un premier temps simplement charger les données en mémoire et observer quelques valeurs.

In [15]:

data_local_path = 'data/'
csv_filename = 'fr.openfoodfacts.org.products.csv'
csv_local_path = data_local_path+csv_filename

if not os.path.isfile(csv_local_path):
    zip_filename = csv_filename+'.zip'
    zip_url = 'https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/parcours-data-scientist/P2/'+zip_filename
    zip_local_path = data_local_path+zip_filename

    r = requests.get(zip_url)
    with open(zip_local_path, 'wb') as f:
        f.write(r.content)

    with zf(zip_local_path, 'r') as zip_file:
        zip_file.extractall(data_local_path)

raw_data = pd.read_csv(csv_local_path, sep='\t')

# Display data types and empty values
raw_data.info()


/home/clement/.local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (0,3,5,19,20,24,25,26,27,28,35,36,37,38,39,48) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320772 entries, 0 to 320771
Columns: 162 entries, code to water-hardness_100g
dtypes: float64(106), object(56)
memory usage: 396.5+ MB


In [55]:
num_rows = len(raw_data.index)
columns_emptiness = pd.DataFrame({
    col : { 
        'count': raw_data[col].isna().sum(),
        'percent': raw_data[col].isna().sum() / num_rows * 100,
    } for col in raw_data.columns
}).transpose().sort_values(by=['count'])

fig = px.bar(columns_emptiness,
    color='percent',
    y='percent',
    labels={
        'index':'column name',
        'percent':'% of empty values',
        'count':'# of empty values',
    },
    hover_data=['count'],
    title='Empty values per column',
    width=1200,
    height=800,
)
fig.show()


In [69]:
clean_data = raw_data.dropna(
        axis='columns',
        thresh=.1 * num_rows
    ).drop_duplicates()

# Display data types and empty values
clean_data.info()

clean_data = clean_data[['product_name',
    'packaging_tags', 'brands_tags', 'manufacturing_places_tags', 'countries_tags',
    'main_category', 'categories_tags', 
    'labels_tags', 'additives_n', 'additives_tags', 
    'nutrition_grade_fr', 'nutrition-score-fr_100g',
    'energy_100g', 'saturated-fat_100g', 'sugars_100g', 'salt_100g', 'sodium_100g',
    'fruits-vegetables-nuts_100g', 'fiber_100g', 'proteins_100g',
]]

# Display statistical summary of each column
clean_data.describe(include="all")


<class 'pandas.core.frame.DataFrame'>
Int64Index: 320772 entries, 0 to 320771
Data columns (total 62 columns):
 #   Column                                   Non-Null Count   Dtype  
---  ------                                   --------------   -----  
 0   code                                     320749 non-null  object 
 1   url                                      320749 non-null  object 
 2   creator                                  320770 non-null  object 
 3   created_t                                320769 non-null  object 
 4   created_datetime                         320763 non-null  object 
 5   last_modified_t                          320772 non-null  object 
 6   last_modified_datetime                   320772 non-null  object 
 7   product_name                             303010 non-null  object 
 8   generic_name                             52795 non-null   object 
 9   quantity                                 104819 non-null  object 
 10  packaging                       

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,...,fiber_100g,proteins_100g,salt_100g,sodium_100g,vitamin-a_100g,vitamin-c_100g,calcium_100g,iron_100g,nutrition-score-fr_100g,nutrition-score-uk_100g
count,3.207490e+05,320749,320770,3.207690e+05,320763,3.207720e+05,320772,303010,52795,104819,...,200886.000000,259922.000000,255510.000000,255463.000000,137554.000000,140867.000000,141050.000000,140462.000000,221210.000000,221210.000000
unique,3.206380e+05,320749,3535,1.896360e+05,189568,1.806390e+05,180495,221347,38584,13826,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,1.111082e+10,http://world-fr.openfoodfacts.org/produit/0036...,usda-ndb-import,1.489056e+09,2017-03-09T10:37:09Z,1.439142e+09,2015-08-09T17:35:42Z,Ice Cream,Pâtes alimentaires au blé dur de qualité supér...,500 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,2.000000e+00,1,169868,2.000000e+01,20,3.300000e+01,33,410,201,4669,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.862111,7.075940,2.028624,0.798815,0.000397,0.023367,0.125163,0.003652,9.165535,9.058049
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12.867578,8.409054,128.269454,50.504428,0.073278,2.236451,3.318263,0.214408,9.055903,9.183589
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-6.700000,-800.000000,0.000000,0.000000,-0.000340,-0.002100,0.000000,-0.000260,-15.000000,-15.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.700000,0.063500,0.025000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.500000,4.760000,0.581660,0.229000,0.000000,0.000000,0.035000,0.001010,10.000000,9.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.600000,10.000000,1.374140,0.541000,0.000107,0.003700,0.106000,0.002400,16.000000,16.000000


In [21]:

# display value frequencies per column
for col in raw_data.columns:
    print(f'\n \
================================================\n \
>    { col }\n \
------------------------------------------------')
    display(raw_data[col].value_counts(dropna=False))



 >    code
 ------------------------------------------------


NaN              23
635646            2
37600105033       2
31200029997       2
16300166360       2
                 ..
3023290102633     1
2441879031357     1
72734127155       1
858246001714      1
3760089530627     1
Name: code, Length: 320639, dtype: int64


 >    url
 ------------------------------------------------


NaN                                                                                                                                             23
http://world-fr.openfoodfacts.org/produit/0070038634102/superior-selection-sparkling-soda-sicilian-lemonade-associated-wholesale-grocers-inc     1
http://world-fr.openfoodfacts.org/produit/0718122066070/organic-ice-cream-three-twins-ice-cream                                                  1
http://world-fr.openfoodfacts.org/produit/0041800354504/100-grape-juice-grape-welch-s                                                            1
http://world-fr.openfoodfacts.org/produit/5601151989814                                                                                          1
                                                                                                                                                ..
http://world-fr.openfoodfacts.org/produit/3256222254272/fromage-frais-nature-u-bio                                    


 >    creator
 ------------------------------------------------


...

 NaN     99562
 0.0     13588
 1.0     11932
 2.0     11083
 14.0    10689
-1.0      8827
 13.0     8409
 12.0     8239
 11.0     8093
 3.0      7620
 20.0     7390
 15.0     6921
 21.0     6650
 16.0     6509
 10.0     6504
 19.0     6476
-2.0      6260
-3.0      6116
 9.0      6085
 17.0     6078
 18.0     5790
 4.0      5687
 8.0      4945
-6.0      4926
-4.0      4819
 23.0     4715
 22.0     4632
 5.0      4555
 6.0      4334
 24.0     4282
-5.0      4211
 7.0      3961
 25.0     2887
 26.0     2643
 27.0     1698
-7.0       963
 28.0      672
-8.0       602
-9.0       315
 29.0      272
 30.0      192
-10.0      157
 33.0      101
-11.0       90
 31.0       76
 32.0       64
-12.0       46
 35.0       34
-13.0       23
 34.0       20
 36.0       17
-14.0        5
 40.0        3
 37.0        2
-15.0        1
 38.0        1
Name: nutrition-score-uk_100g, dtype: int64


 >    glycemic-index_100g
 ------------------------------------------------


NaN    320772
Name: glycemic-index_100g, dtype: int64


 >    water-hardness_100g
 ------------------------------------------------


NaN    320772
Name: water-hardness_100g, dtype: int64

In [13]:
clean_data = raw_data

clean_data = clean_data.dropna(axis='columns', how='all')
# Display data types and empty values
clean_data.info()

clean_data = clean_data.dropna(axis='index', how='all')
# Display data types and empty values
clean_data.info()

clean_data = clean_data.dropna(axis='columns', how='all')
# Display data types and empty values
clean_data.info()

clean_data = clean_data.dropna(axis='index', how='all')
# Display data types and empty values
clean_data.info()

clean_data = clean_data.dropna(axis='columns', how='all')
# Display data types and empty values
clean_data.info()

clean_data = clean_data.dropna(axis='index', how='all')
# Display data types and empty values
clean_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320772 entries, 0 to 320771
Columns: 146 entries, code to nutrition-score-uk_100g
dtypes: float64(90), object(56)
memory usage: 357.3+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 320772 entries, 0 to 320771
Columns: 146 entries, code to nutrition-score-uk_100g
dtypes: float64(90), object(56)
memory usage: 359.8+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 320772 entries, 0 to 320771
Columns: 146 entries, code to nutrition-score-uk_100g
dtypes: float64(90), object(56)
memory usage: 359.8+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 320772 entries, 0 to 320771
Columns: 146 entries, code to nutrition-score-uk_100g
dtypes: float64(90), object(56)
memory usage: 359.8+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 320772 entries, 0 to 320771
Columns: 146 entries, code to nutrition-score-uk_100g
dtypes: float64(90), object(56)
memory usage: 359.8+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 320772 ent

In [ ]:

# display first 5 rows
clean_data.head()


In [18]:

raw_data = pd.read_csv(csv_local_path, 
    sep='\t',
    usecols=
        lambda column_name: 
            column_name not in ['code', 'url', 'creator', 'created_t', 'created_datetime', 'last_modified_t', 'last_modified_datetime',
                'image_url', 'image_small_url', 'nutrition-score-uk_100g ']
)

# display first 5 rows
raw_data.head()


/home/clement/.local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (19,20,24,25,26,27,28,35,36,37,38,39,48) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,product_name,generic_name,quantity,packaging,packaging_tags,brands,brands_tags,categories,categories_tags,categories_fr,...,ph_100g,fruits-vegetables-nuts_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
0,Farine de blé noir,NaN,1kg,NaN,NaN,Ferme t'y R'nao,ferme-t-y-r-nao,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Banana Chips Sweetened (Whole),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,14.0,14.0,NaN,NaN
2,Peanuts,NaN,NaN,NaN,NaN,Torn & Glasser,torn-glasser,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
3,Organic Salted Nut Mix,NaN,NaN,NaN,NaN,Grizzlies,grizzlies,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,12.0,NaN,NaN
4,Organic Polenta,NaN,NaN,NaN,NaN,Bob's Red Mill,bob-s-red-mill,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:

# Display data types and empty values
raw_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320772 entries, 0 to 320771
Columns: 153 entries, product_name to water-hardness_100g
dtypes: float64(106), object(47)
memory usage: 374.4+ MB


In [20]:

# Display statistical summary of each column
raw_data.describe(include="all")


,product_name,generic_name,quantity,packaging,packaging_tags,brands,brands_tags,categories,categories_tags,categories_fr,...,ph_100g,fruits-vegetables-nuts_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
count,303010,52795,104819,78960,78961,292360,292352,84410,84389,84411,...,49.000000,3036.000000,165.000000,948.000000,0.0,268.000000,221210.000000,221210.000000,0.0,0.0
unique,221347,38584,13826,14547,12064,58784,50253,36982,21142,21152,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,Ice Cream,Pâtes alimentaires au blé dur de qualité supér...,500 g,Carton,"sachet,plastique",Carrefour,carrefour,"Snacks sucrés,Biscuits et gâteaux,Biscuits","en:sugary-snacks,en:biscuits-and-cakes,en:bisc...","Snacks sucrés,Biscuits et gâteaux,Biscuits",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,410,201,4669,2153,3959,2978,3149,301,802,802,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.425698,31.458587,15.412121,49.547785,NaN,341.700764,9.165535,9.058049,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.047841,31.967918,3.753028,18.757932,NaN,425.211439,9.055903,9.183589,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,8.000000,6.000000,NaN,0.000000,-15.000000,-15.000000,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.300000,0.000000,12.000000,32.000000,NaN,98.750000,1.000000,1.000000,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.200000,23.000000,15.000000,50.000000,NaN,195.750000,10.000000,9.000000,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.400000,51.000000,15.000000,64.250000,NaN,383.200000,16.000000,16.000000,NaN,NaN


Nous voyons que, pour chaque arbre listé, nous disposons des informations suivantes (la description des colonnes est disponible sur le site [OpenData](https://opendata.paris.fr/explore/dataset/les-arbres/information/)) :
- `id` : simple identifiant de l'arbre (entier, ex. : `99874`)
- `type_emplacement` : type de l'emplacement (texte, ex. : `"Arbre"`)
- `domanialite` : type de lieu auquel appartient l'arbre (texte, ex. : `"Jardin"`)
- `arrondissement` : arrondissement de Paris où est situé l'arbre (texte, ex. : `"PARIS 7E ARRDT"`)
- `complement_addresse` : complement d'adress (texte, pas d'exemple visible)
- `numero` : numéro de l'adress (texte, pas d'exemple visible)
- `lieu` : adresse de l'arbre (texte, ex. : `"MAIRIE DU 7E 116 RUE DE GRENELLE PARIS 7E"`)
- `id_emplacement` : identifiant de l'emplacement (texte, ex. : `"19"`)
- `libelle_francais` : nom commun (vernaculaire) de l'espèce de l'arbre (texte, ex. : `"Marronnier"`)
- `genre` : genre de l'arbre (texte, ex. : `"Aesculus"`)
- `espece` : espèce de l'arbre (texte, ex. : `"hippocastanum"`)
- `variete` : variété de l'arbre (texte, pas d'exemple visible)
- `circonference_cm` : circonférence en centimètres de l'arbre (entier, ex. : `20`)
- `hauteur_m` : taille en mètres de l'arbre (entier, ex. : `5`)
- `stade_developpement` : stade de développement de l'arbre (texte, ex. : `"A"` pour "Adulte")
- `remarquable` : si l'arbre est "remarquable" ou non (booléen, ex. : `0` pour un arbre "non remarquable")
- `geo_point_2d_a` : latitude de la position de l'arbre (nombre à virgule, ex. : `48.857620`)
- `geo_point_2d_b` : longitude de la position de l'arbre (nombre à virgule, ex. : `2.320962`)

Nous voyons déjà que parmis les quelques premières données :
- un certain certain nombre de valeurs ne sont pas fournies (`NaN` = "Not a Number" = donnée non disponible)
- nous pouvons classer les variables selon leur type :
    - quantitatives
        - discrètes : `id`, `circonference_cm`, `hauteur_m`
        - continues : `geo_point_2d_a`, `geo_point_2d_b`
    - qualitatives
        - nominales : `type_emplacement`, `domanialite`, `arrondissement`, `complement_addresse`, `numero`, `lieu`, `id_emplacement`, `libelle_francais`, `genre`, `espece`, `variete`
        - ordinales : `stade_developpement`, `remarquable`
- on peut aussi les classer en trois grandes catégories, d'après leur sens :
    - métadonnées internes au système : `id`, `id_emplacement`, `type_emplacement`
    - données de localisation : `arrondissement`, `complement_addresse`, `numero`, `lieu`, `geo_point_2d_a`, `geo_point_2d_b`
    - données de description : 
        - taille : `circonference_cm`, `hauteur_m` et `stade_developpement`
        - type : `libelle_francais`, `genre`, `espece` et `variete`
        - autre : `remarquable`


Nous allons observer plus précisément les types de valeurs et les valeurs vides :

Nous voyons ici que la plupart des arbres sont des platanes adultes.
Cette information permet d'optimiser les achats et le stockage du materiel et des produits adaptés spécifiquement à l'entretien de ces arbres.

---

_[Licence GPL-v3](https://github.com/fleuryc/oc_ingenieur-ia_P2-Participez-a-un-concours-sur-la-Smart-City/blob/main/LICENSE)_
